# 下載財報

參考資料：

* [Python 財報爬蟲](https://goo.gl/rVZhZa)
* [請問有高手會匯入這個網頁的資料到excel嗎? ](https://www.mobile01.com/topicdetail.php?f=291&t=3360713)

In [5]:
import requests
import pandas as pd
import numpy as np

In [17]:
def financial_statement(year, season, exchange='sii', type='綜合損益表'):
    if year > 1911:
        year -= 1911
        
    if type == '綜合損益表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif type == '資產負債表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif type == '營益分析查詢彙總表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb06'
    else:
        print('type does not match')

    # 一些參數：
    # TYPEK => 市場別
    # sii>上市
    # otc>上櫃
    # rotc>興櫃
    # pub>公開發行

    # year => 年度
    # season => 季別
    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK': exchange,
        'year': year,
        'season': season,
    })
    
    r.encoding = 'utf8'
    
    dfs = pd.read_html(r.text)
    
    
    for i, df in enumerate(dfs):
        df.columns = df.iloc[0]
        dfs[i] = df.iloc[1:]
        
    df = pd.concat(dfs).applymap(lambda x: x if x != '--' else np.nan)
    df = df[df['公司代號'] != '公司代號']
    df = df[~df['公司代號'].isnull()]
    if any(df.columns.str.contains("合計：")):
        df = df.loc[:, ~df.columns.str.contains("合計：")]
    return df

In [20]:
df = financial_statement(2017, 2, type='營益分析查詢彙總表')

In [21]:
df[df['公司名稱']=='台泥']

,公司代號,公司名稱,毛利率(%)(營業毛利)/(營業收入),營業利益率(%)(營業利益)/(營業收入),營業收入(百萬元),稅前純益率(%)(稅前純益)/(營業收入),稅後純益率(%)(稅後純益)/(營業收入)
1,1101,台泥,18.01,13.08,45893.85,12.54,9.23
